# Considering Bias in Data
This notebook will explore the bias in Wikipedias page rating system. Using article data of various political leaders from all around the world, we query the [ORES system](https://ores.wikimedia.org/) in order to obtain rating estimates. 

## License
This code example was developed by Nathan Grant for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided un
der the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.0 - May 13, 2022

## Imports

In [1]:
# 
# These are standard python modules
import json, time, urllib.parse, os
#
# The 'requests' module is not a standard Python module. You will need to install this with pip/pip3 if you do not already have it
import requests
# The 'pandas' module is not a standard Python module. You will need to install this with pip/pip3 if you do not already have it
import pandas as pd

## Tabular Data Parsing

In [2]:
# Handle data inconsistencies in wikipedia data
politicians_dataframe = pd.read_csv(os.getcwd()+ '\\data\\politicians_by_country_SEPT2022.csv')
politicians_dataframe.head()

,name,url,country
0,Shahjahan Noori,https://en.wikipedia.org/wiki/Shahjahan_Noori,Afghanistan
1,Abdul Ghafar Lakanwal,https://en.wikipedia.org/wiki/Abdul_Ghafar_Lak...,Afghanistan
2,Majah Ha Adrif,https://en.wikipedia.org/wiki/Majah_Ha_Adrif,Afghanistan
3,Haroon al-Afghani,https://en.wikipedia.org/wiki/Haroon_al-Afghani,Afghanistan
4,Tayyab Agha,https://en.wikipedia.org/wiki/Tayyab_Agha,Afghanistan


In [3]:
# Check for duplicate records
politicians_dataframe['name'].value_counts()

Torokul Dzhanuzakov           4
Lazar Tomanović               2
Abdirahman Aw Ali Farrah      2
Juraj Košút                   2
José Francisco Barrundia      2
                             ..
Verner Lindberg               1
Murilo Melo Filho             1
José Antonio Velutini         1
Voldemārs Ozols               1
Maslah Mohammed Siad Barre    1
Name: name, Length: 7534, dtype: int64

In [4]:
politicians_dataframe = politicians_dataframe.drop_duplicates(subset='name')

In [5]:
population_dataframe = pd.read_csv(os.getcwd()+ '\\data\\population_by_country_2022.csv')
population_dataframe.head()

,Geography,Population (millions)
0,WORLD,7963.0
1,AFRICA,1419.0
2,NORTHERN AFRICA,251.0
3,Algeria,44.9
4,Egypt,103.5


In [6]:
# Get rows that are regions instead of countries
region_mask = population_dataframe['Geography'].str.isupper()
regions = population_dataframe[region_mask]

# Isolate countries
countries = population_dataframe[~region_mask]

## Obtaining revision ids

In [7]:
#########
#
#    CONSTANTS
#

# The basic English Wikipedia API endpoint
API_ENWIKIPEDIA_ENDPOINT = "https://en.wikipedia.org/w/api.php"

# We'll assume that there needs to be some throttling for these requests - we should always be nice to a free data resource
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': '<uwnetid@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2022',
}

# This is a string of additional page properties that can be returned see the Info documentation for
# what can be included. If you don't want any this can simply be the empty string
PAGEINFO_EXTENDED_PROPERTIES = "talkid|url|watched|watchers"
#PAGEINFO_EXTENDED_PROPERTIES = ""

# This template lists the basic parameters for making this
PAGEINFO_PARAMS_TEMPLATE = {
    "action": "query",
    "format": "json",
    "titles": "",           # to simplify this should be a single page title at a time
    "prop": "info",
    "inprop": PAGEINFO_EXTENDED_PROPERTIES
}


The API request will be made using one procedure. The idea is to make this reusable. The procedure is parameterized, but relies on the constants above for the important parameters. The underlying assumption is that this will be used to request data for a set of article pages. Therefore the parameter most likely to change is the article_title.

In [8]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_pageinfo_per_article(article_title = None, 
                                 endpoint_url = API_ENWIKIPEDIA_ENDPOINT, 
                                 request_template = PAGEINFO_PARAMS_TEMPLATE,
                                 headers = REQUEST_HEADERS):
    # Make sure we have an article title
    if not article_title: return None
    
    request_template['titles'] = article_title
        
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or any other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(endpoint_url, headers=headers, params=request_template)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

In [9]:
# Gather wiki responses for each politician and add them to a list
responses = []
for name in politicians_dataframe['name']:
    responses.append(request_pageinfo_per_article(name))

In [10]:
# Filter responses and display unscessful queries
unsuccessful_queries = []
successful_queries = []
for r in responses:
    if "lastrevid" not in r['query']['pages'][list(r['query']['pages'].keys())[0]]:
        unsuccessful_queries.append(r['query']['pages'][list(r['query']['pages'].keys())[0]]['title'])
    else:
        data = r['query']['pages'][list(r['query']['pages'].keys())[0]]
        successful_queries.append((data['title'],data['lastrevid']))

In [11]:
unsuccessful_queries

['Prince Ofosu Sefah',
 'Harjit Kaur Talwandi',
 'Abd al-Razzaq al-Hasani',
 'Kang Sun-nam',
 'Abiodun Abimbola Orekoya',
 'Segun “Aeroland” Adewale',
 'Roman Konoplev',
 'Nhlanhla “Lux” Dlamini']

## Obtaining page ratings

We are going to use the wp10 model because its the only model that had a wiki outlining its [performance](https://meta.wikimedia.org/wiki/Objective_Revision_Evaluation_Service/wp10) on the test data

In [12]:
#########
#
#    CONSTANTS
#

# The current ORES API endpoint
API_ORES_SCORE_ENDPOINT = "https://ores.wikimedia.org/v3"
# A template for mapping to the URL
API_ORES_SCORE_PARAMS = "/scores/{context}/{revid}/{model}"

# Use some delays so that we do not hammer the API with our requests
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': '<uwnetid@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2022'
}

# A dictionary of English Wikipedia article titles (keys) and sample revision IDs that can be used for this ORES scoring example
ARTICLE_REVISIONS = { 'Bison':1085687913 , 'Northern flicker':1086582504 , 'Red squirrel':1083787665 , 'Chinook salmon':1085406228 , 'Horseshoe bat':1060601936 }

# This template lists the basic parameters for making an ORES request
ORES_PARAMS_TEMPLATE = {
    "context": "enwiki",        # which WMF project for the specified revid
    "revid" : "",               # the revision to be scored - this will probably change each call
    "model": "wp10"   # the AI/ML scoring model to apply to the reviewion
}
#
# The current ML models for English wikipedia are:
#   "articlequality"
#   "articletopic"
#   "damaging"
#   "version"
#   "draftquality"
#   "drafttopic"
#   "goodfaith"
#   "wp10"
#
# The specific documentation on these is scattered so if you want to use one you'll have to look around.
#

In [13]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_ores_score_per_article(article_revid = None, 
                                   endpoint_url = API_ORES_SCORE_ENDPOINT, 
                                   endpoint_params = API_ORES_SCORE_PARAMS, 
                                   request_template = ORES_PARAMS_TEMPLATE,
                                   headers = REQUEST_HEADERS,
                                   features=False):
    # Make sure we have an article revision id
    if not article_revid: return None
    
    # set the revision id into the template
    request_template['revid'] = article_revid
    
    # now, create a request URL by combining the endpoint_url with the parameters for the request
    request_url = endpoint_url+endpoint_params.format(**request_template)
    
    # the features used by the ML model can sometimes be returned as well as scores
    if features:
        request_url = request_url+"?features=true"
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like ORES - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

In [14]:
# Iterate through all of the revids and get the article scores
score_responses = []
for name, revid in successful_queries:
    response = request_ores_score_per_article(revid)
    score_responses.append((name, response))

In [15]:
# Example score response
score_responses[0]

('Shahjahan Noori',
 {'enwiki': {'models': {'wp10': {'version': '0.9.2'}},
   'scores': {'1099689043': {'wp10': {'score': {'prediction': 'GA',
       'probability': {'B': 0.15912710013204634,
        'C': 0.3317754589473183,
        'FA': 0.029544640342677096,
        'GA': 0.43829652594886387,
        'Start': 0.034888134844429625,
        'Stub': 0.006368139784664634}}}}}}})

In [16]:
# Parse score responses

In [18]:
# Filter through the responses to determine which ones were valid
unsuccessful_score_queries = []
successful_score_queries = []
for name, res in score_responses:
    revid = list(res['enwiki']['scores'].keys())[0]
    # If the wp10 element is in the response dictionary
    if 'wp10' in res['enwiki']['scores'][revid]:
        score = res['enwiki']['scores'][revid]['wp10']['score']
        # If prediction isnt in the json
        if 'prediction' not in score:
            # Add it to unsuccessful queries
            unsuccessful_score_queries.append(name)
        else:
            #  Else add it to successful queries
            if score['prediction'] is not "NaN":
                successful_score_queries.append((name,revid,score['prediction']))
            else:
                unsuccessful_score_queries.append(name)
    else:
        unsuccessful_score_queries.append(name)

In [19]:
# No unsuccessful queries
len(unsuccessful_score_queries)

0

In [20]:
# Create a dataframe of the successful queries with their scores
score_df = pd.DataFrame(data=successful_score_queries,columns=['name','revid','score'])
score_df.head()

,name,revid,score
0,Shahjahan Noori,1099689043,GA
1,Abdul Ghafar Lakanwal,943562276,Start
2,Majah Ha Adrif,852404094,Start
3,Haroon al-Afghani,1095102390,B
4,Tayyab Agha,1104998382,Start


In [21]:
# Remove politicians whos records could not be resolved
politicians_dataframe = politicians_dataframe[~politicians_dataframe["name"].isin(unsuccessful_queries)]
politicians_dataframe.head(2)

,name,url,country
0,Shahjahan Noori,https://en.wikipedia.org/wiki/Shahjahan_Noori,Afghanistan
1,Abdul Ghafar Lakanwal,https://en.wikipedia.org/wiki/Abdul_Ghafar_Lak...,Afghanistan


In [22]:
# Combine the politician table with the articles scores
merged_df = pd.merge(score_df,politicians_dataframe, how='inner', on='name')
merged_df.head()

,name,revid,score,url,country
0,Shahjahan Noori,1099689043,GA,https://en.wikipedia.org/wiki/Shahjahan_Noori,Afghanistan
1,Abdul Ghafar Lakanwal,943562276,Start,https://en.wikipedia.org/wiki/Abdul_Ghafar_Lak...,Afghanistan
2,Majah Ha Adrif,852404094,Start,https://en.wikipedia.org/wiki/Majah_Ha_Adrif,Afghanistan
3,Haroon al-Afghani,1095102390,B,https://en.wikipedia.org/wiki/Haroon_al-Afghani,Afghanistan
4,Tayyab Agha,1104998382,Start,https://en.wikipedia.org/wiki/Tayyab_Agha,Afghanistan


In [23]:
## scheme to generate regions from the countries
geography_map = []
levels = []

# Filter through countries regions
for g,pop in population_dataframe.values:
    # If its upper case then add it to the stack of region names
    if g.isupper():
        levels.append(g)
    # If its not then get the last region on the stack and add it to the list
    else:
        geography_map.append([g,levels[-1],pop])

In [24]:
# Example country tuple
geography_map[0], geography_map[50]

(['Algeria', 'NORTHERN AFRICA', 44.9], ['Gabon', 'MIDDLE AFRICA', 2.4])

In [25]:
# Crete a datafraome of the countries with their regions and populations
region_df = pd.DataFrame(data=geography_map,columns=["country","region","population"])
region_df.head()

,country,region,population
0,Algeria,NORTHERN AFRICA,44.9
1,Egypt,NORTHERN AFRICA,103.5
2,Libya,NORTHERN AFRICA,6.8
3,Morocco,NORTHERN AFRICA,36.7
4,Sudan,NORTHERN AFRICA,46.9


In [26]:
#Merge the regions and populations with the article ratings table
final_df = pd.merge(region_df,merged_df, how='outer', on='country')
# Select necessary columns
final_df = final_df[["country","region","population","name","revid","score"]]
# Reset the column names to the desired table output
final_df.columns = ["country","region","population","article_title","revision_id","article_quality"]
# Remove rows that have NAs
has_nans = final_df[final_df['population'].isna()]
# Final df is all articles that a rating could be generated for
final_df = final_df[~final_df['population'].isna()]
final_df.head()

,country,region,population,article_title,revision_id,article_quality
0,Algeria,NORTHERN AFRICA,44.9,Said Abadou,1112193748,Stub
1,Algeria,NORTHERN AFRICA,44.9,Tahar Allan,1059626268,Stub
2,Algeria,NORTHERN AFRICA,44.9,Mohamed Seghir Babes,1079379844,Stub
3,Algeria,NORTHERN AFRICA,44.9,Djelloul Baghli,1053461392,Stub
4,Algeria,NORTHERN AFRICA,44.9,Noureddine Bahbouh,1099284595,Stub


In [27]:
# Save to disk in the working folder
final_df.to_csv(os.getcwd()+"\\wp_politicians_by_country.csv",index=False)

In [28]:
# Find out which countries couldnt be resolved
all_nans = has_nans['country'].unique()
all_nans

array(['Korean'], dtype=object)

Here there is no data for Korean because in the population CSV, the countries are represented as Korea, North and Korea, South so therefore the table operations result in no key matches. For this reason we will not include these two countries in the analysis

In [29]:
# Save contries that dont have a match to a text tfile
with open(os.getcwd()+"\\wp_countries-no_match.txt",'a') as f:
    for c in all_nans:
        f.write(c+"\n")

## Analysis

Here we will calculate total-articles-per-population (a ratio representing the number of articles per person) and high-quality-articles-per-population (a ratio representing the number of high quality articles per person) on a country-by-country and regional basis. Also we will normalize them all to be “per capita”.

Top 10 countries by coverage: 
- The 10 countries with the highest total articles per capita (in descending order):

In [30]:
# Group articles together by country and get a count of articles
total_articles_per_country = final_df.groupby("country")['article_title'].count()
total_articles_per_country.head()

country
Afghanistan    118
Albania         83
Algeria         34
Andorra         10
Angola          42
Name: article_title, dtype: int64

In [33]:
# Merge together the articles for each country with countries populations
total_articles_per_pop_country = pd.merge(total_articles_per_country,countries, 
                                          how="inner",left_index=True ,right_on="Geography")
# Create a new column that is the number of articles divided by the normalized population
total_articles_per_pop_country["articles_per_population_mil"] = total_articles_per_pop_country["article_title"]/\
                                                        total_articles_per_pop_country["Population (millions)"]
# Select desired columns select the largest and sort in descending order
total_articles_per_pop_country_highest = total_articles_per_pop_country[["Geography","articles_per_population_mil"]].sort_values(ascending=False, 
                                        by ="articles_per_population_mil")
# Display the top 10
total_articles_per_pop_country_highest.head(10)

,Geography,articles_per_population_mil
185,Monaco,inf
226,Palau,inf
211,San Marino,inf
183,Liechtenstein,inf
231,Tuvalu,inf
223,Nauru,inf
78,Antigua and Barbuda,170.000000
217,Federated States of Micronesia,130.000000
201,Andorra,100.000000
80,Barbados,93.333333


Here the top 6 are inf because their popualtion values are 0.

Below is the table with those values removed

In [36]:
total_articles_per_pop_country_highest.iloc[6:,:].head(10)

,Geography,articles_per_population_mil
78,Antigua and Barbuda,170.000000
217,Federated States of Micronesia,130.000000
201,Andorra,100.000000
80,Barbados,93.333333
222,Marshall Islands,90.000000
208,Montenegro,60.000000
41,Seychelles,60.000000
184,Luxembourg,52.857143
138,Bhutan,51.250000
85,Grenada,50.000000


Bottom 10 countries by coverage: 
- The 10 countries with the lowest total articles per capita (in ascending order) .

In [38]:
# Select desired columns select the largest and sort in ascending order
total_articles_per_pop_country_lowest = total_articles_per_pop_country[["Geography","articles_per_population_mil"]]. \
                                        sort_values(ascending=True, 
                                        by ="articles_per_population_mil")
total_articles_per_pop_country_lowest.head(10)

,Geography,articles_per_population_mil
60,eSwatini,0.0
57,Sao Tome and Principe,0.0
146,Brunei,0.0
159,"China, Hong Kong SAR",0.0
160,"China, Macao SAR",0.0
39,Reunion,0.0
90,Puerto Rico,0.0
152,Philippines,0.0
82,Curacao,0.0
225,New Zealand,0.0


Top 10 countries by high quality:
- The 10 countries with the highest high quality articles per capita (in descending order)

In [60]:
# Filter articles that are rated FA or GA by the model
only_quality_articles = final_df[(final_df['article_quality'] == "FA") | (final_df['article_quality'] == "GA")]
only_quality_articles.head()

,country,region,population,article_title,revision_id,article_quality
60,Libya,NORTHERN AFRICA,6.8,Khalifa Haftar,1115568588,FA
73,Libya,NORTHERN AFRICA,6.8,Fayez al-Sarraj,1115566067,GA
115,Morocco,NORTHERN AFRICA,36.7,Muhammad al-Kattani,1088340779,GA
160,Sudan,NORTHERN AFRICA,46.9,Abdallahi ibn Muhammad,1092462251,GA
192,Tunisia,NORTHERN AFRICA,11.8,Rached Ghannouchi,1109470504,GA


In [61]:
# Merge together the articles for each country with countries populations
total_articles_per_country = only_quality_articles.groupby("country")['article_title'].count()
total_qual_articles_per_pop_country = pd.merge(total_articles_per_country,countries, 
                                          how="inner",left_on="country" ,right_on="Geography")
# Create a new column that is the number of articles divided by the normalized population
total_qual_articles_per_pop_country["articles_per_population_mil"] = total_qual_articles_per_pop_country["article_title"]/\
                                                        total_qual_articles_per_pop_country["Population (millions)"]
# Select desired columns select the largest and sort in descending order
total_qual_articles_per_pop_country_high = total_qual_articles_per_pop_country[["Geography",
                                        "articles_per_population_mil"]].sort_values(ascending=False, 
                                        by ="articles_per_population_mil")
total_qual_articles_per_pop_country_high.head(10)

,Geography,articles_per_population_mil
85,Tuvalu,inf
2,Andorra,20.000000
53,Montenegro,5.000000
1,Albania,2.142857
9,Bosnia-Herzegovina,1.470588
49,Lithuania,1.071429
19,Croatia,1.052632
74,Slovenia,0.952381
61,Palestinian Territory,0.925926
28,Gabon,0.833333


Here the top one is inf because their popualtion values are 0.

Below is the table with those values removed

In [48]:
total_qual_articles_per_pop_country_high.iloc[1:,:].head(10)

,Geography,articles_per_population_mil
2,Andorra,20.000000
53,Montenegro,5.000000
1,Albania,2.142857
9,Bosnia-Herzegovina,1.470588
49,Lithuania,1.071429
19,Croatia,1.052632
74,Slovenia,0.952381
61,Palestinian Territory,0.925926
28,Gabon,0.833333
25,Estonia,0.769231


Bottom 10 countries by high quality: 
- The 10 countries with the lowest high quality articles per capita (in ascending order).

In [49]:
# Select desired columns select the largest and sort in ascending order
total_qual_articles_per_pop_country_low = total_qual_articles_per_pop_country[["Geography",
                                        "articles_per_population_mil"]].sort_values(ascending=True, 
                                        by ="articles_per_population_mil")
total_qual_articles_per_pop_country_low.head(10)

,Geography,articles_per_population_mil
35,India,0.004234
83,Thailand,0.014970
39,Japan,0.016013
58,Nigeria,0.018307
90,Vietnam,0.020121
17,Colombia,0.020367
86,Uganda,0.021186
60,Pakistan,0.021204
79,Sudan,0.021322
37,Iran,0.022573


Geographic regions by total coverage: 
- A rank ordered list of geographic regions (in descending order) by total articles per capita.

In [50]:
# Aggregate counts together by region
article_counts_region = final_df.groupby("region")['article_quality'].count()
article_counts_region.head()

region
CARIBBEAN          201
CENTRAL AMERICA    193
CENTRAL ASIA       103
EAST ASIA          245
EASTERN AFRICA     646
Name: article_quality, dtype: int64

In [51]:
# Merge together the articles for each region with regions populations
total_articles_per_pop_reg = pd.merge(article_counts_region,regions, 
                                          how="inner",left_index=True ,right_on="Geography")
# Create a new column that is the number of articles divided by the normalized population
total_articles_per_pop_reg["articles_per_population_mil"] = total_articles_per_pop_reg["article_quality"]/\
                                                        total_articles_per_pop_reg["Population (millions)"]
# Select desired columns and sort values in descending order
total_articles_per_pop_reg = total_articles_per_pop_reg[["Geography",
                                        "articles_per_population_mil"]].sort_values(ascending=False, 
                                        by ="articles_per_population_mil")
# Set column names
total_articles_per_pop_reg.columns = ["country","articles_per_population_mil"]
# Display
total_articles_per_pop_reg

,country,articles_per_population_mil
199,SOUTHERN EUROPE,5.788079
77,CARIBBEAN,4.568182
178,WESTERN EUROPE,3.543147
188,EASTERN EUROPE,2.526132
167,NORTHERN EUROPE,2.429907
110,WESTERN ASIA,2.326531
215,OCEANIA,1.954545
58,SOUTHERN AFRICA,1.695652
27,EASTERN AFRICA,1.365751
95,SOUTH AMERICA,1.327189


Geographic regions by high quality coverage: 
- Rank ordered list of geographic regions (in descending order) by high quality articles per capita.


In [52]:
# Aggregate counts together by region
article_counts_region_good = only_quality_articles.groupby("region")['article_quality'].count()
# Display
article_counts_region_good.head()

region
CARIBBEAN           8
CENTRAL AMERICA    10
CENTRAL ASIA        3
EAST ASIA          16
EASTERN AFRICA     15
Name: article_quality, dtype: int64

In [54]:
article_counts_region_good

region
CARIBBEAN           8
CENTRAL AMERICA    10
CENTRAL ASIA        3
EAST ASIA          16
EASTERN AFRICA     15
EASTERN EUROPE     38
MIDDLE AFRICA       5
NORTHERN AFRICA     7
NORTHERN EUROPE     8
OCEANIA             2
SOUTH AMERICA      12
SOUTH ASIA         23
SOUTHEAST ASIA     24
SOUTHERN AFRICA     4
SOUTHERN EUROPE    46
WESTERN AFRICA     13
WESTERN ASIA       28
WESTERN EUROPE     22
Name: article_quality, dtype: int64

In [53]:
# Merge together the articles for each region with regions populations
total_articles_per_pop_reg_good = pd.merge(article_counts_region_good,regions, 
                                          how="inner",left_index=True ,right_on="Geography")
# Create a new column that is the number of articles divided by the normalized population
total_articles_per_pop_reg_good["articles_per_population_mil"] = total_articles_per_pop_reg_good["article_quality"]/\
                                                        total_articles_per_pop_reg_good["Population (millions)"]
# Select desired columns and sort values in descending order
total_articles_per_pop_reg_good = total_articles_per_pop_reg_good[["Geography",
                                        "articles_per_population_mil"]].sort_values(ascending=False, 
                                        by ="articles_per_population_mil")
# Set column names
total_articles_per_pop_reg_good.columns = ["country","articles_per_population_mil"]
total_articles_per_pop_reg

,country,articles_per_population_mil
199,SOUTHERN EUROPE,5.788079
77,CARIBBEAN,4.568182
178,WESTERN EUROPE,3.543147
188,EASTERN EUROPE,2.526132
167,NORTHERN EUROPE,2.429907
110,WESTERN ASIA,2.326531
215,OCEANIA,1.954545
58,SOUTHERN AFRICA,1.695652
27,EASTERN AFRICA,1.365751
95,SOUTH AMERICA,1.327189
